# Debugging why CIS-TTC comparison results in ~2x processed data #134

In [8]:
from psycopg2 import connect
import configparser
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.ticker as ticker
import folium

CONFIG = configparser.ConfigParser()
CONFIG.read('db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

from IPython.display import HTML

def print_table(sql, con):
    return HTML(pandasql.read_sql(sql, con).to_html(index=False))

## Looking at GTFS Data 

The counts for the query below for one day of the route 514 is never more than 102. So the maximum count over 7 days for any segment of a route should be 714. Looking at the data from the ttc and our analysis, the TTC seems to be fairly accurate with their counts (it could be undercounting due to filtering out some trips). Some of our general counts are very high, some are ever about 1100 for the week. So it seems that our data processing is overcounting. 

The highest count for the 504 is 259 per day, so the total count for 7 days should not exceed 1813. The TTC processed data does not exceed 1813, but our processed data can have counts as high as over 4500. 

```sql
WITH to_stop_table AS (
SELECT stop_id to_stop, t.trip_id, direction_id, arrival_time
FROM crosic.stop_times_20171119_20171125 s LEFT JOIN crosic.trips_20171119_20171125 t ON s.trip_id = t.trip_id
INNER JOIN crosic.routes_20171119_20171125 r ON t.route_id = r.route_id
WHERE route_short_name = '514' AND s.stop_id IN (SELECT to_stop_id FROM crosic.section_runs)), 

from_stop_table AS (
SELECT stop_id from_stop, t.trip_id, direction_id, departure_time
FROM crosic.stop_times_20171119_20171125 s LEFT JOIN crosic.trips_20171119_20171125 t ON s.trip_id = t.trip_id
INNER JOIN crosic.routes_20171119_20171125 r ON t.route_id = r.route_id
WHERE route_short_name = '514' AND s.stop_id IN (SELECT from_stop_id FROM crosic.section_runs)
), 

stops AS (
SELECT to_stop, from_stop, t.direction_id, t.trip_id, arrival_time, departure_time 
FROM from_stop_table f JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
)

SELECT to_stop, from_stop, direction_id, 
AVG(EXTRACT(EPOCH FROM arrival_time - departure_time)) /60 AS time_diff_minutes_gtfs, COUNT(*) cnt
FROM stops
WHERE arrival_time > departure_time
GROUP BY to_stop, from_stop, direction_id; 
```

## Compare trip counts of temporary tables from count queries

There might be an issue with how the data is being counted, since the `trip_id` counts between the TTC processed CIS data and our processed CIS data are very different (our data has up to 4x more trips than the TTC data). 

The first step in investigating if my queries that count the number of trips in our CIS processed data are correct is to look at the first part of the queries and look at the temporary tables created in the query. 

An example of a count query (found in `comparing_ttc_data` notebook) that counts the overall number of trips between the segments is: 
```sql 
WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_504_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_504_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs)
), 

stops AS (
SELECT to_stop, from_stop, t.direction_id, t.trip_id, arrival_time, departure_time -- , 
-- AVG(EXTRACT(EPOCH FROM arrival_time - departure_time)) /60 AS time_diff_minutes_our_cis
FROM from_stop_table f JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
WHERE arrival_time > departure_time and from_stop <> to_stop
-- GROUP BY to_stop, from_stop, t.direction_id
),

ttc_cis AS (
SELECT DISTINCT fromstopname, tostopname, directionid, COUNT(*) cnt, to_stop_id, from_stop_id, routenumber,
AVG(EXTRACT(EPOCH FROM (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(toarrstoptime, 'HH24:MI:SS'))::timestamp
- (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(fromstopdepaturetime, 'HH24:MI:SS'))::timestamp)  ) /60
AS time_diff_minutes_ttc
FROM section_runs
WHERE routenumber = 504 and toarrstoptime > fromstopdepaturetime
GROUP BY fromstopname, tostopname, to_stop_id, from_stop_id, directionid, routenumber
ORDER BY time_diff_minutes_ttc
)

SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt cnt_ttc, time_diff_minutes_ttc, 
COUNT(*) cnt_our_cis , AVG(EXTRACT(EPOCH FROM arrival_time - departure_time)) /60 AS time_diff_minutes_our_cis
FROM stops s JOIN ttc_cis ttc ON s.to_stop = ttc.to_stop_id AND s.from_stop = ttc.from_stop_id 
WHERE s.direction_id = ttc.directionid and ttc.routenumber = 504 -- and arrival_time > departure_time
GROUP BY to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, ttc.cnt, time_diff_minutes_ttc, routenumber
ORDER BY time_diff_minutes_ttc, time_diff_minutes_our_cis; 
```

To investigate this possibility further, the following queries count how many trips are being examined from both the TTC and the CIS dataset. Basically, my queries that count the number of trips that occur use multiple temporary tables to get their count numbers. Some temporary tables are created from from our processed CIS data, and some are created from the TTC's processed CIS data. The queries below look at the number of trips that are examined later in the overall count query as a result of both the temporary tables that pull from our processed data and the TTC's processed data.

A relatively similar number of trips are being pulled from the temporary queries. 

#### Route 504

In [2]:
# our processed data
# look at to_stop_table, from_stop_table, and stops temporary tables

sql_debug1_504 = '''

WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_504_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs WHERE routenumber = 504)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_504_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs WHERE routenumber = 504)
)

SELECT COUNT(DISTINCT t.trip_id) 
FROM from_stop_table f INNER JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
WHERE from_stop <> to_stop; 
'''

pandasql.read_sql(sql_debug1_504, con)

,count
0,3551


In [3]:
# TTC processed data 
# look at ttc_cis temp table

sql_debug2_504 = '''

SELECT COUNT(DISTINCT ah_tripid)
FROM section_runs
WHERE routenumber = 504 AND from_stop_id <> to_stop_id; 
'''

pandasql.read_sql(sql_debug2_504, con)

,count
0,3724


#### Route 514

In [9]:
# our processed data 
# look at to_stop_table, from_stop_table, and stops temporary tables

sql_debug1_514 = '''

WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_514_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs WHERE routenumber = 514)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_514_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs WHERE routenumber = 514)
)

SELECT COUNT(DISTINCT t.trip_id) 
FROM from_stop_table f INNER JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
WHERE from_stop <> to_stop; 
'''

pandasql.read_sql(sql_debug1_514, con)

,count
0,1275


In [8]:
# TTC processed data 
# look at ttc_cis temp table

sql_debug2_514 = '''

SELECT COUNT(DISTINCT ah_tripid)
FROM section_runs
WHERE routenumber = 514 AND from_stop_id <> to_stop_id; 
'''

pandasql.read_sql(sql_debug2_514, con)

,count
0,1351


## Count volume of data produced for each processed November day with previously processed weekday of data

### Check `match_stop` table data 

Find the volume of data for the weekday processed on October 4 for both the 514 route and the 504 route

In [13]:
sql_count_514 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514; 
'''
pandasql.read_sql(sql_count_514, con)

,count
0,4436


In [27]:
sql_count_504 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504; 
'''
pandasql.read_sql(sql_count_504, con)

,count
0,24701


### Route 514
Examine each day from Novemeber 19 to Novemeber 25 for the 514 route to find their counts (note: November 19th is a Sunday and November 25th is a Saturday so their counts might be a bit different). 

The expected count is ~4500 per day on weekdays, but we are seeing that the count for a day with the November data is ~8000 on average. 

In [17]:
# November 19
sql_count_514_20171119 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514_nov
WHERE arrival_time >= '2017-11-19'::date AND arrival_time <= '2017-11-20'::date 
AND  departure_time >= '2017-11-19'::date AND departure_time <= '2017-11-20'::date;
'''

pandasql.read_sql(sql_count_514_20171119, con)

,count
0,4352


In [18]:
# November 20
sql_count_514_20171120 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514_nov
WHERE arrival_time >= '2017-11-20'::date AND arrival_time <= '2017-11-21'::date 
AND  departure_time >= '2017-11-20'::date AND departure_time <= '2017-11-21'::date;
'''

pandasql.read_sql(sql_count_514_20171120, con)

,count
0,8069


In [19]:
# November 21
sql_count_514_20171121 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514_nov
WHERE arrival_time >= '2017-11-21'::date AND arrival_time <= '2017-11-22'::date 
AND  departure_time >= '2017-11-21'::date AND departure_time <= '2017-11-22'::date;
'''

pandasql.read_sql(sql_count_514_20171121, con)

,count
0,7924


In [20]:
# November 22
sql_count_514_20171122 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514_nov
WHERE arrival_time >= '2017-11-22'::date AND arrival_time <= '2017-11-23'::date 
AND  departure_time >= '2017-11-22'::date AND departure_time <= '2017-11-23'::date;
'''

pandasql.read_sql(sql_count_514_20171122, con)

,count
0,8165


In [23]:
# November 23
sql_count_514_20171123 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514_nov
WHERE arrival_time >= '2017-11-23'::date AND arrival_time <= '2017-11-24'::date 
AND  departure_time >= '2017-11-23'::date AND departure_time <= '2017-11-24'::date;
'''

pandasql.read_sql(sql_count_514_20171123, con)

,count
0,7847


In [24]:
# November 24
sql_count_514_20171124 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514_nov
WHERE arrival_time >= '2017-11-24'::date AND arrival_time <= '2017-11-25'::date 
AND  departure_time >= '2017-11-24'::date AND departure_time <= '2017-11-25'::date;
'''

pandasql.read_sql(sql_count_514_20171124, con)

,count
0,7621


In [26]:
# November 25
sql_count_514_20171125 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_514_nov
WHERE arrival_time >= '2017-11-25'::date AND arrival_time <= '2017-11-26'::date 
AND  departure_time >= '2017-11-25'::date AND departure_time <= '2017-11-26'::date;
'''

pandasql.read_sql(sql_count_514_20171125, con)

,count
0,5774


### Route 504 
Check the counts of the 504 data to see if there is a similar pattern of being about double the amount of data. 

We expected to see ~25 000 records for an average weekday in November. However, the November data has ~50 000 records for an average weekday. 

In [28]:
# November 19
sql_count_504_20171119 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504_nov
WHERE arrival_time >= '2017-11-19'::date AND arrival_time <= '2017-11-20'::date 
AND  departure_time >= '2017-11-19'::date AND departure_time <= '2017-11-20'::date;
'''

pandasql.read_sql(sql_count_504_20171119, con)

,count
0,24949


In [29]:
# November 20
sql_count_504_20171120 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504_nov
WHERE arrival_time >= '2017-11-20'::date AND arrival_time <= '2017-11-21'::date 
AND  departure_time >= '2017-11-20'::date AND departure_time <= '2017-11-21'::date;
'''

pandasql.read_sql(sql_count_504_20171120, con)

,count
0,53830


In [30]:
# November 21
sql_count_504_20171121 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504_nov
WHERE arrival_time >= '2017-11-21'::date AND arrival_time <= '2017-11-22'::date 
AND  departure_time >= '2017-11-21'::date AND departure_time <= '2017-11-22'::date;
'''

pandasql.read_sql(sql_count_504_20171121, con)

,count
0,55612


In [31]:
# November 22
sql_count_504_20171122 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504_nov
WHERE arrival_time >= '2017-11-22'::date AND arrival_time <= '2017-11-23'::date 
AND  departure_time >= '2017-11-22'::date AND departure_time <= '2017-11-23'::date;
'''

pandasql.read_sql(sql_count_504_20171122, con)

,count
0,54558


In [32]:
# November 23
sql_count_504_20171123 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504_nov
WHERE arrival_time >= '2017-11-23'::date AND arrival_time <= '2017-11-24'::date 
AND  departure_time >= '2017-11-23'::date AND departure_time <= '2017-11-24'::date;
'''

pandasql.read_sql(sql_count_504_20171123, con)

,count
0,53128


In [33]:
# November 24
sql_count_504_20171124 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504_nov
WHERE arrival_time >= '2017-11-24'::date AND arrival_time <= '2017-11-25'::date 
AND  departure_time >= '2017-11-24'::date AND departure_time <= '2017-11-25'::date;
'''

pandasql.read_sql(sql_count_504_20171124, con)

,count
0,52385


In [34]:
# November 25
sql_count_504_20171125 = '''
SELECT COUNT(*) 
FROM crosic.match_stop_504_nov
WHERE arrival_time >= '2017-11-25'::date AND arrival_time <= '2017-11-26'::date 
AND  departure_time >= '2017-11-25'::date AND departure_time <= '2017-11-26'::date;
'''

pandasql.read_sql(sql_count_504_20171125, con)

,count
0,33516


### Check CIS angle tables 

From the analysis of both of the routes, there must have been some type of mistake in the processing of the 7 days of data. To see if the mistake occured when creating the `match_stop` tables, check the counts of the `dd_cis_514_angle`, `dd_cis_504_angle`, `cis_nov_504_angle` and the `cis_nov_514_angle` tables.  

The counts are very similar in these two tables, and there is a decrease in counts on the weekdends. Since these tables were the CIS data tables that were used to create the `match_stop` tables, and their counts are accurate, there must have been some type of mistake when creating the `match_stop` tables. 

#### Route 514

In [37]:
# count from weekday in October
sql_count_514 = '''
SELECT COUNT(*) 
FROM crosic.dd_cis_514_angle; 
'''
pandasql.read_sql(sql_count_514, con)

,count
0,23204


In [40]:
# check a weekday - November 23
sql_count_514 = '''
SELECT COUNT(*) 
FROM crosic.cis_nov_514_angle
WHERE date_time >= '2017-11-23'::date AND date_time <= '2017-11-24'::date; 
'''
pandasql.read_sql(sql_count_514, con)

,count
0,23637


In [39]:
# check a weekday - November 22
sql_count_514 = '''
SELECT COUNT(*) 
FROM crosic.cis_nov_514_angle
WHERE date_time >= '2017-11-22'::date AND date_time <= '2017-11-23'::date; 
'''
pandasql.read_sql(sql_count_514, con)

,count
0,23437


In [41]:
# check a weekend day - November 25
sql_count_514 = '''
SELECT COUNT(*) 
FROM crosic.cis_nov_514_angle
WHERE date_time >= '2017-11-25'::date AND date_time <= '2017-11-26'::date; 
'''
pandasql.read_sql(sql_count_514, con)

,count
0,17985


#### Route 504

In [48]:
# count from weekday in October 
sql_count_504 = '''
SELECT COUNT(*) 
FROM crosic.dd_cis_504_angle; 
'''
pandasql.read_sql(sql_count_504, con)

,count
0,111555


In [46]:
# weekday count - Nov 22
sql_count_504 = '''
SELECT COUNT(*) 
FROM crosic.cis_nov_504_angle
WHERE date_time >= '2017-11-22'::date AND date_time <= '2017-11-23'::date; 
'''
pandasql.read_sql(sql_count_504, con)

,count
0,119039


In [47]:
# weekday count - Nov 24
sql_count_504 = '''
SELECT COUNT(*) 
FROM crosic.cis_nov_504_angle
WHERE date_time >= '2017-11-24'::date AND date_time <= '2017-11-25'::date; 
'''
pandasql.read_sql(sql_count_504, con)

,count
0,119682


In [45]:
# weekend count - Nov 25
sql_count_504 = '''
SELECT COUNT(*) 
FROM crosic.cis_nov_504_angle
WHERE date_time >= '2017-11-25'::date AND date_time <= '2017-11-26'::date; 
'''
pandasql.read_sql(sql_count_504, con)

,count
0,76154


Now that we know that the mistake is in the query that created the `match_stop` table, we can examine the query to find potential mistakes. Right now, the query looks like this: 

```sql
DROP SEQUENCE IF EXISTS stops; 
CREATE SEQUENCE stops START 100; 

SELECT nextval('stops'); 

WITH line_data AS(
SELECT geom AS line, direction_id FROM shapes_geom_20171119_20171125
    INNER JOIN trips_20171119_20171125 USING (shape_id)
    WHERE shape_id IN (695251, 695252, 695253, 695254)
    GROUP BY line, shape_id, direction_id
    ORDER BY shape_id
),

cis_gtfs AS(
SELECT date_time, id AS cis_id, stop_id, vehicle, a.direction_id,
ST_LineLocatePoint(line, position) AS cis_to_line,
ST_LineLocatePoint(line, geom) AS stop_to_line,
(CASE WHEN ST_LineLocatePoint(line, position) > ST_LineLocatePoint(line, geom)
      THEN 'after'
      WHEN ST_LineLocatePoint(line, position) < ST_LineLocatePoint(line, geom)
      THEN 'before'
      WHEN ST_LineLocatePoint(line, position) = ST_LineLocatePoint(line, geom)
      THEN 'same'
      END) AS line_position,
ST_Distance(position::geography, geom::geography) AS distance
FROM line_data a, cis_nov_514_angle b
INNER JOIN stops_20171119_20171125 USING (stop_id)
WHERE a.direction_id = b.direction_id
ORDER BY vehicle, a.direction_id, date_time
),

stop_orders AS (
SELECT *,
(CASE WHEN lag(stop_id, 1) OVER (PARTITION BY vehicle ORDER BY date_time) IS NULL
      THEN nextval('stops')
      WHEN stop_id <> lag(stop_id, 1) OVER (PARTITION BY vehicle ORDER BY date_time)
      THEN nextval('stops')
      WHEN stop_id = lag(stop_id, 1) OVER (PARTITION BY vehicle ORDER BY date_time)
      THEN currval('stops')
END) AS stop_order
FROM cis_gtfs
WHERE (line_position = 'before' AND distance <= 200) OR (line_position = 'after' AND distance <= 10) OR (line_position = 'same' AND distance <= 100)
ORDER BY vehicle, direction_id, date_time
)

SELECT MIN(date_time) AS arrival_time, MAX(date_time) AS departure_time, vehicle, stop_id, direction_id, array_agg(DISTINCT cis_id) AS cis_group
INTO match_stop_514_nov
FROM stop_orders
GROUP BY stop_order, vehicle, stop_id, direction_id; 
```

Notice how there are more than two shape_id's in the list in the `line_data` temporary table. This is a problem since there should only be one shape in each direction in that list. So try the same query but with only 2 shapes: the longest shape in each direction. Use the following queries to find these shape_id's: 


```sql 
SELECT DISTINCT MAX(ST_LENGTH(geom)), s.shape_id, direction_id
FROM crosic.shapes_geom_20171119_20171125 c
RIGHT JOIN
(SELECT DISTINCT shape_id, direction_id FROM crosic.trips_20171119_20171125 
WHERE trip_headsign LIKE '%504%') s ON c.shape_id = s.shape_id
GROUP BY s.shape_id, direction_id
ORDER BY MAX(ST_LENGTH(geom)) DESC; 



SELECT DISTINCT MAX(ST_LENGTH(geom)), s.shape_id, direction_id
FROM crosic.shapes_geom_20171119_20171125 c
RIGHT JOIN
(SELECT DISTINCT shape_id, direction_id FROM crosic.trips_20171119_20171125 
WHERE trip_headsign LIKE '%514%') s ON c.shape_id = s.shape_id
GROUP BY s.shape_id, direction_id
ORDER BY MAX(ST_LENGTH(geom)) DESC; 

```

**longest shape_ids in each direction**

514: 695251, 695253

504: 695081, 695070

After re-running the process with only 2 shape_id's, the counts in the `match_stop` file and the queries in the `comparing_ttc_data` notebook were around what we expected and were not extremely high. 

## Results from first fix 

Below are the general counts (i.e. including weekends and non-rush hour times) for both the 504 and 514 route. 

In [9]:
sql_result = '''
WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_514_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs)
OR t.stop_id IN (SELECT to_stop_id2 FROM crosic.section_runs)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_514_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs)
OR t.stop_id IN (SELECT from_stop_id2 FROM crosic.section_runs)
), 

stops AS (
SELECT to_stop, from_stop, t.direction_id, t.trip_id, arrival_time, departure_time 
FROM from_stop_table f JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
),

ttc_cis AS (
SELECT DISTINCT fromstopname, tostopname, directionid, COUNT(*) cnt, to_stop_id, from_stop_id, routenumber,
AVG(EXTRACT(EPOCH FROM (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(toarrstoptime, 'HH24:MI:SS'))::timestamp
- (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(fromstopdepaturetime, 'HH24:MI:SS'))::timestamp)  ) /60
AS time_diff_minutes_ttc
FROM section_runs
WHERE routenumber = 514 and toarrstoptime > fromstopdepaturetime
GROUP BY fromstopname, tostopname, to_stop_id, from_stop_id, directionid, routenumber
ORDER BY time_diff_minutes_ttc
),

output AS (
SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt cnt_ttc, time_diff_minutes_ttc, 
COUNT(*) cnt_our_cis,

AVG(EXTRACT(EPOCH FROM arrival_time - departure_time)) /60
AS time_diff_minutes_our_cis


FROM stops s JOIN ttc_cis ttc ON s.to_stop = ttc.to_stop_id AND s.from_stop = ttc.from_stop_id 

WHERE s.direction_id = ttc.directionid and ttc.routenumber = 514 and arrival_time > departure_time
GROUP BY to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, ttc.cnt, time_diff_minutes_ttc, routenumber
ORDER BY time_diff_minutes_ttc, time_diff_minutes_our_cis
) 

SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt_ttc, time_diff_minutes_ttc, 
cnt_our_cis, time_diff_minutes_our_cis, abs(time_diff_minutes_our_cis - time_diff_minutes_ttc) ttc_our_time_difference
FROM output
WHERE cnt_ttc > 100
ORDER BY abs(cnt_ttc - cnt_our_cis) DESC; 
'''
pandasql.read_sql(sql_result, con)

,routenumber,to_stop_id,from_stop_id,fromstopname,tostopname,directionid,cnt_ttc,time_diff_minutes_ttc,cnt_our_cis,time_diff_minutes_our_cis,ttc_our_time_difference
0,514,15439,8207,KING ST EAST AT PARLIAMENT ST,DISTILLERY LOOP,0,586,5.620051,8,304.708333,299.088282
1,514,1389,15439,DISTILLERY LOOP,KING ST EAST AT PARLIAMENT ST,1,569,4.569420,40,125.833333,121.263913
2,514,6113,4341,KING ST WEST AT DUFFERIN ST,DUFFERIN GATE LOOP,1,589,4.083192,346,9.243738,5.160546
3,514,1897,3070,KING ST WEST AT YONGE ST,KING ST EAST AT JARVIS ST,0,570,3.271988,359,3.536722,0.264734
4,514,7211,436,KING ST WEST AT SPADINA AVE,KING ST WEST AT BATHURST ST,1,612,2.032135,422,8.292259,6.260124
5,514,3070,1845,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT YONGE ST,0,577,3.094743,394,3.121827,0.027084
6,514,2253,6113,DUFFERIN GATE LOOP,KING ST WEST AT BATHURST ST,0,590,11.164972,430,14.103101,2.938129
7,514,5334,1389,KING ST EAST AT PARLIAMENT ST,KING ST EAST AT JARVIS ST,1,613,3.551930,490,7.493197,3.941267
8,514,4341,7211,KING ST WEST AT BATHURST ST,KING ST WEST AT DUFFERIN ST,1,601,8.511370,478,12.766388,4.255018
9,514,436,3357,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT SPADINA AVE,1,591,4.782854,474,9.049191,4.266337


In [5]:
sql_result = '''
WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_504_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs)
OR t.stop_id IN (SELECT to_stop_id2 FROM crosic.section_runs)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_504_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs)
OR t.stop_id IN (SELECT from_stop_id2 FROM crosic.section_runs)
), 

stops AS (
SELECT to_stop, from_stop, t.direction_id, t.trip_id, arrival_time, departure_time 
FROM from_stop_table f JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
),

ttc_cis AS (
SELECT DISTINCT fromstopname, tostopname, directionid, COUNT(*) cnt, to_stop_id, from_stop_id, routenumber,
AVG(EXTRACT(EPOCH FROM (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(toarrstoptime, 'HH24:MI:SS'))::timestamp
- (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(fromstopdepaturetime, 'HH24:MI:SS'))::timestamp)  ) /60
AS time_diff_minutes_ttc
FROM section_runs
WHERE routenumber = 504 and toarrstoptime > fromstopdepaturetime
GROUP BY fromstopname, tostopname, to_stop_id, from_stop_id, directionid, routenumber
ORDER BY time_diff_minutes_ttc
),

output AS (
SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt cnt_ttc, time_diff_minutes_ttc, 
COUNT(*) cnt_our_cis,

AVG(EXTRACT(EPOCH FROM arrival_time - departure_time)) /60
AS time_diff_minutes_our_cis


FROM stops s JOIN ttc_cis ttc ON s.to_stop = ttc.to_stop_id AND s.from_stop = ttc.from_stop_id 

WHERE s.direction_id = ttc.directionid and ttc.routenumber = 504 and arrival_time > departure_time
GROUP BY to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, ttc.cnt, time_diff_minutes_ttc, routenumber
ORDER BY time_diff_minutes_ttc, time_diff_minutes_our_cis
) 

SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt_ttc, time_diff_minutes_ttc, 
cnt_our_cis, time_diff_minutes_our_cis, abs(time_diff_minutes_our_cis - time_diff_minutes_ttc) ttc_our_time_difference
FROM output
WHERE cnt_ttc > 105
ORDER BY abs(cnt_ttc - cnt_our_cis) DESC; 
'''
pandasql.read_sql(sql_result, con)

,routenumber,to_stop_id,from_stop_id,fromstopname,tostopname,directionid,cnt_ttc,time_diff_minutes_ttc,cnt_our_cis,time_diff_minutes_our_cis,ttc_our_time_difference
0,504,13209,4341,KING ST WEST AT DUFFERIN ST,DUNDAS WEST STATION AT BAY 4,1,1470,19.864626,32,19.510417,0.354209
1,504,13050,8207,KING ST EAST AT PARLIAMENT ST,BROADVIEW STATION AT BAY 6,0,1508,15.112533,170,48.378431,33.265898
2,504,1389,13050,BROADVIEW STATION AT BAY 6,KING ST EAST AT PARLIAMENT ST,1,1501,15.544937,505,18.766997,3.222060
3,504,1897,3070,KING ST WEST AT YONGE ST,KING ST EAST AT JARVIS ST,0,1561,3.083280,1042,4.495218,1.411938
4,504,3070,1845,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT YONGE ST,0,1562,3.221084,1106,4.101552,0.880468
5,504,7211,436,KING ST WEST AT SPADINA AVE,KING ST WEST AT BATHURST ST,1,1686,2.059134,1387,4.120896,2.061762
6,504,4341,7211,KING ST WEST AT BATHURST ST,KING ST WEST AT DUFFERIN ST,1,1661,8.396338,1486,8.949742,0.553404
7,504,436,3357,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT SPADINA AVE,1,1599,4.953096,1433,6.124215,1.171119
8,504,5334,1389,KING ST EAST AT PARLIAMENT ST,KING ST EAST AT JARVIS ST,1,1634,3.471440,1488,3.912399,0.440959
9,504,4748,2253,KING ST WEST AT BATHURST ST,KING ST WEST AT SPADINA AVE,0,1695,3.450157,1597,3.710499,0.260342


From the numbers above, it seems that our processing and the TTC processing is not exactly the same. There is a significant difference between our count and the TTC count. Also, the counts for segments that include subway stations is still really high. 

In [6]:
con.close()